<a href="https://colab.research.google.com/github/ThijsVercammen/Masterproef/blob/main/resnet50_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installeer nodige tools

In [ ]:
!pip install tflite-support
!pip install onnxruntime
!pip install -U tf2onnx
#!pip install git+https://github.com/onnx/tensorflow-onnx
!pip install tensorflow-datasets
!wget https://machinelearningmastery.com/wp-content/uploads/2019/03/zebra.jpg

Importeer de nodige bibliotheken

In [ ]:
import numpy as np
import time
import os

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime
from datetime import datetime

from tensorflow.keras.utils import plot_model


Inladen van het ResNet50 model en dit converteren naar TFLite

In [ ]:
model = tf.keras.applications.resnet50.ResNet50()
model.save('./model')
model.save("mod.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open('model.tflite', 'wb').write(tflite_model)

#plot_model(model, to_file='model.png')

Test model op testafbeelding

In [ ]:
from PIL import Image

im = Image.open("./zebra.jpg")
res_im = im.resize((416, 416))
np_res_im = np.array(res_im, dtype=np.float32)

now = datetime.now()

img_array = tf.expand_dims(np_res_im, 0) # Create a batch
now = datetime.now()
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions)
#print(predictions)
print(datetime.now()-now)

print(
    "This image most likely belongs to {}."
    .format([np.argmax(predictions)])
)

0:00:01.410698
This image most likely belongs to [340].


Test TFLite model op testafbeelding

In [ ]:
from PIL import Image

im = Image.open("zebra.jpg")
interpreter = tf.lite.Interpreter(model_path="./model.tflite")
interpreter.allocate_tensors()
det = interpreter.get_signature_list()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

res_im = im.resize((416, 416))
input_data = np.asarray(res_im, dtype=np.float32)

interpreter.set_tensor(input_details[0]['index'], [input_data])
now = datetime.now()
interpreter.invoke()
print(datetime.now()-now)
output_data = interpreter.get_tensor(output_details[0]['index'])
#print(output_data)

print(np.argmax(output_data))

0:00:00.448454
340


Voeg Metadata toe aan het model
( labels.txt met imagenet labels moet eerst worden geupload)

In [ ]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

ImageClassifierWriter = image_classifier.MetadataWriter
model_p = "./model.tflite" # TFLite model
label_p = "./labels.txt" # label file voor label formaat
save_p = "./model_meta.tflite" # opslaan pad
input_norm_mean = 0.0
input_norm_std = 1.0

# metadata scrijver
writer = ImageClassifierWriter.create_for_inference(
    writer_utils.load_file(model_p), [input_norm_mean], [input_norm_std],
    [label_p])

# Voeg metadata aan het model toe en sla op
writer_utils.save_file(writer.populate(), save_p)

Converteer naar ONNX via standaard of TFLite model

In [ ]:
!python -m tf2onnx.convert --saved-model ./model --output model.onnx
#!python -m tf2onnx.convert --tflite ./model.tflite --output model_lite.onnx

Test ONNX model met testafbeelding

In [ ]:
import onnx
import onnxruntime as ort
im = Image.open("/content/zebra.jpg")
res_im = im.resize((416, 416))
input_data = np.asarray(res_im, dtype=np.float32)

print(input_data.shape)
#a = tf.reshape(input_data, [1,160,160,3])
ort_session = ort.InferenceSession("model.onnx")
now = datetime.now()
outputs = ort_session.run(
    None,
    {"input_1": [input_data]},
)
print(datetime.now()-now)
print(np.argmax(outputs))


(224, 224, 3)
0:00:00.142570
340


Evalueer het Standaard, TFLite en ONNX model

In [ ]:
import tensorflow_datasets as tfds
from tensorflow_datasets.core import dataset_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import onnx
import onnxruntime as ort

ds_test, info = tfds.load('imagenet_v2', split='test',  with_info=True)
interpreter = tf.lite.Interpreter(model_path="./model.tflite")
ort_session = ort.InferenceSession("model.onnx")

interpreter.allocate_tensors()
det = interpreter.get_signature_list()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#tfds.as_dataframe(ds_test.take(4), info)
val1 = 0
val2 = 0
val3 = 0

examples = list(dataset_utils.as_numpy(ds_test.take(9990)))
for i, ex in enumerate(examples):
  # process input data
  a = tf.keras.layers.CenterCrop(224, 224)(ex['image'])
  x = np.expand_dims(a, axis=0)
  y = np.copy(x)
  z = preprocess_input(y)

  # voorspel top 1 resultaat
  predicted_batch = model.predict(z)
  res1 = np.argmax(predicted_batch)

  interpreter.set_tensor(input_details[0]['index'], z)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  res2 = np.argmax(output_data)

  outputs = ort_session.run(None, {"input_1": z})
  res3 = np.argmax(outputs)

  #print("{} - {}".format(res2, ex['label']))
  # vergelijk labels
  if res1 == ex['label']:
    val1 += 1
  if res2 == ex['label']:
    val2 += 1
  if res3 == ex['label']:
    val3 += 1

print("Standaard model precisie:  {}".format(val1))
print("TFLite model precisie:     {}".format(val2))
print("ONNX model precisie:       {}".format(val3))
#predicted_id = tf.math.argmax(predicted_batch, axis=-1)
#predicted_label_batch = class_names[predicted_id]
#print(predicted_label_batch)